Codigo para obtener las tablas del PDF

In [2]:
import pdfplumber
import pandas as pd

# Configuración para extracción más precisa
custom_settings = {
    "vertical_strategy": "lines",
    "horizontal_strategy": "lines",
    "intersection_tolerance": 1
}

# Lista con los nombres de las pestañas (hojas) que sí quieres guardar
hojas_deseadas = ["Pag1_Tab2", "Pag2_Tab2"]

# Archivo PDF
pdf_path = "ficha_tecnica.pdf"
tablas_filtradas = []

with pdfplumber.open(pdf_path) as pdf:
    for i, pagina in enumerate(pdf.pages):
        texto = pagina.extract_text()
        tablas = pagina.extract_tables(table_settings=custom_settings)

        for j, tabla in enumerate(tablas):
            nombre_hoja = f"Pag{i+1}_Tab{j+1}"

            if nombre_hoja in hojas_deseadas:
                df = pd.DataFrame(tabla)

                # Aplicar encabezado si hay varias filas
                if df.shape[0] > 1:
                    df.columns = df.iloc[0]
                    df = df[1:]
                    df.reset_index(drop=True, inplace=True)

                # Guardar solo si coincide con los nombres deseados
                tablas_filtradas.append({
                    "nombre_hoja": nombre_hoja,
                    "data": df
                })

                # Mostrar en consola
                print(f"\n✅ Mostrando tabla {nombre_hoja}:")
                print(df)

# Guardar solo las tablas seleccionadas
with pd.ExcelWriter("tablas_seleccionadas_bbva.xlsx", engine="openpyxl") as writer:
    for tabla in tablas_filtradas:
        hoja = tabla["nombre_hoja"]
        tabla["data"].to_excel(writer, sheet_name=hoja, index=False)

print(f"\n📂 {len(tablas_filtradas)} tablas guardadas en 'tablas_seleccionadas_bbva.xlsx'")


✅ Mostrando tabla Pag1_Tab2:
0                                                      \
0                                                       
1                                                       
2           Fecha de Inicio de Operaciones 13/04/2009   
3                                                       
4       Número de Unidades en Circulación 114,473,519   
5                                                       
6                                                       
7                                                       
8                                                       
9                                                       
10                                                      
11                                                      
12                                                      
13                                                      
14                                                      
15                                                      
1

Codigo de VB para ejecutar la macro que organice la información de las tablas (Abrir un excel y ejecutar la macro)

In [ ]:
Sub PDF_BBVA2_Universal()

    Dim libroOrigen As Workbook
    Dim libroDestino As Workbook
    Dim hojaNueva As Worksheet

    ' Nombre del archivo generado por Python
    Dim nombreLibroOrigen As String
    nombreLibroOrigen = "tablas_seleccionadas_bbva.xlsx"

    ' Establecer referencias
    Set libroDestino = ThisWorkbook ' donde corre la macro
    Set hojaNueva = libroDestino.Sheets.Add(After:=libroDestino.Sheets(libroDestino.Sheets.Count))

    ' Abrir el archivo origen si no está abierto
    On Error Resume Next
    Set libroOrigen = Workbooks(nombreLibroOrigen)
    If libroOrigen Is Nothing Then
        Set libroOrigen = Workbooks.Open(libroDestino.Path & "\" & nombreLibroOrigen)
    End If
    On Error GoTo 0

    ' Copiar de Pag1_Tab2
    With libroOrigen.Sheets("Pag1_Tab2")
        .Range("B5").Copy hojaNueva.Range("A1") ' Valor unidad
        .Range("E3").Copy hojaNueva.Range("D1") ' Fecha
        .Range("H2:K3").Copy hojaNueva.Range("D2") ' Tabla resumen
        .Range("G8:K12").Copy hojaNueva.Range("J2") ' Comisiones
        hojaNueva.Range("J1:N1").Merge
        hojaNueva.Range("J1").Value = "Comisiones"
    End With

    ' Copiar de Pag2_Tab2
    With libroOrigen.Sheets("Pag2_Tab2")
        .Range("B3:K9").Copy hojaNueva.Range("Q1")
    End With

    ' Limpiar columnas no deseadas en tabla desde Pag2_Tab2
    hojaNueva.Range("U2:U7").Delete Shift:=xlToLeft
    hojaNueva.Range("V2:V7").Delete Shift:=xlToLeft
    hojaNueva.Range("W2:W7").Delete Shift:=xlToLeft
    hojaNueva.Range("Q4:W4").Delete Shift:=xlUp
    hojaNueva.Range("Q1:W1").Merge
    hojaNueva.Range("Q1").HorizontalAlignment = xlCenter

    ' Ajustar anchos
    hojaNueva.Columns("A:A").AutoFit
    hojaNueva.Columns("D:G").AutoFit
    hojaNueva.Columns("J:N").AutoFit
    hojaNueva.Columns("Q:W").AutoFit

    ' Aplicar bordes a zonas claves
    Dim r1 As Range, r2 As Range, r3 As Range
    Set r1 = hojaNueva.Range("A1")
    Set r2 = hojaNueva.Range("D1:G3")
    Set r3 = hojaNueva.Range("J1:N6")

    With r1.Borders
    .LineStyle = xlContinuous
    .Weight = xlThin
    .ColorIndex = 0
    End With

    With r2.Borders
        .LineStyle = xlContinuous
        .Weight = xlThin
        .ColorIndex = 0
    End With

    With r3.Borders
        .LineStyle = xlContinuous
        .Weight = xlThin
        .ColorIndex = 0
    End With

    With hojaNueva.Range("Q1:W6").Borders
        .LineStyle = xlContinuous
        .Weight = xlThin
        .ColorIndex = 0
    End With


    MsgBox "✅ Datos organizados exitosamente en '" & hojaNueva.Name & "'", vbInformation

End Sub
